In [16]:
from glob import glob
import pandas as pd

In [14]:
files_dict = {filename.split('/')[-1].split('.')[0]: filename for filename in glob('results/orf_x_ref/*')}
dfs = {key: pd.read_csv(file_path) for key, file_path in files_dict.items()}

In [24]:
dfs['rdrp_orf1_orf_x_ref'].sort_values(by='identity_score').head(20)

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
5968,SRR8658352,NODE_1127_length_3722_cov_5479.529767_SRR8658352,NODE_1127_length_3722_cov_5479.529767_SRR86583...,NC_001556.1_rdrp_orf1_[Tobacco_mild_green_mosa...,0.219677,49,0.897219,579,742,1196,827,1111
6540,SRR8727441,NODE_296_length_3822_cov_10.632844_SRR8727441,NODE_296_length_3822_cov_10.632844_SRR8727441_3,NC_008310.2_rdrp_orf1_[Hibiscus_latent_Singapo...,0.234266,64,0.878199,657,858,1253,977,1134
3954,SRR5892088,NODE_2635_length_2222_cov_118.392247_SRR5892088,NODE_2635_length_2222_cov_118.392247_SRR5892088_4,OQ789506.1_rdrp_orf1_[Bertoki_tobamovirus_1],0.234767,39,0.754054,427,558,955,740,773
6985,SRR9596414,NODE_199_length_2867_cov_7.196416_SRR9596414,NODE_199_length_2867_cov_7.196416_SRR9596414_11,NC_008614.1_rdrp_orf1_[Cucumber_mottle_virus],0.235294,53,0.883616,598,782,1269,885,1166
688,SRR10613610,NODE_2497_length_2241_cov_11.267098_SRR10613610,NODE_2497_length_2241_cov_11.267098_SRR10613610_4,OQ789506.1_rdrp_orf1_[Bertoki_tobamovirus_1],0.236234,38,0.754692,430,563,956,746,773
2946,SRR4435987,NODE_1426_length_3115_cov_410.357472_SRR4435987,NODE_1426_length_3115_cov_410.357472_SRR4435987_1,DQ356949.1_rdrp_orf1_[Maracuja_mosaic_virus],0.240046,54,0.896939,668,879,1204,980,1103
569,SRR10255703,NODE_9835_length_2717_cov_4.322727_SRR10255703,NODE_9835_length_2717_cov_4.322727_SRR10255703_1,NC_009642.1_rdrp_orf1_[Bell_pepper_mottle_virus],0.240113,50,0.908858,538,708,1187,779,1116
6071,SRR8658353,NODE_1178_length_3181_cov_2591.634987_SRR8658353,NODE_1178_length_3181_cov_2591.634987_SRR86583...,OR350232.1_rdrp_orf1_[Piper_chlorosis_virus],0.240169,49,0.941799,541,712,1162,756,1118
6682,SRR8727443,NODE_345_length_3431_cov_8.403995_SRR8727443,NODE_345_length_3431_cov_8.403995_SRR8727443_3,NC_008310.2_rdrp_orf1_[Hibiscus_latent_Singapo...,0.240949,60,0.890000,608,801,1233,900,1134
2237,SRR2086713,NODE_49_length_6528_cov_15.603121_SRR2086713,NODE_49_length_6528_cov_15.603121_SRR2086713_7,OR350232.1_rdrp_orf1_[Piper_chlorosis_virus],0.242344,49,0.886659,569,751,1214,847,1118


In [20]:
dfs['rdrp_orf2_orf_x_ref']

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
0,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_001728.1_rdrp_orf2_[Odontoglossum_ringspot_...,0.387387,13,0.892761,204,333,498,373,458
1,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_009642.1_rdrp_orf2_[Bell_pepper_mottle_virus],0.382090,11,0.898123,207,335,502,373,464
2,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_040685.2_rdrp_orf2_[Opuntia_virus_2],0.379538,9,0.812332,188,303,382,373,312
3,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_016519.1_rdrp_orf2_[Clitoria_yellow_mottle_...,0.348974,12,0.914209,222,341,467,373,435
4,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_001556.1_rdrp_orf2_[Tobacco_mild_green_mosa...,0.355623,10,0.882038,212,329,538,373,494
...,...,...,...,...,...,...,...,...,...,...,...,...
28750,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,AJ238607.1_rdrp_orf2_[Potato_mop-top_virus],0.264789,29,0.855422,261,355,559,415,499
28751,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,KT225277.1_rdrp_orf2_[Soil-borne_virus_2],0.271429,30,0.843373,255,350,543,415,478
28752,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,X14006.1_rdrp_orf2_[Pea_early_browning_virus],0.270186,22,0.775904,235,322,564,415,471
28753,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,L23972.1_rdrp_orf2_[Pepper_ringspot_virus],0.286164,26,0.766265,227,318,567,415,470
